## 模块

模块中的定义可以导入到其他模块或者主模块

模块是一个包含 Python 定义和语句的文件
- 文件名就是模块名后跟文件后缀 `.py`

在一个模块内部，模块名可以通过全局变量 `__name__` 获得

导入模块后，可以用模块名访问函数

In [1]:
import fibo    # 导入模块

fibo.fib(1000)

0 1 1 2 3 5 8 13 21 34 55 89 144 233 377 610 987 


In [2]:
fibo.fib2(1000)

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987]

In [3]:
# 模块名
fibo.__name__

'fibo'

In [4]:
# 函数赋值给局部变量，别名（alias）
fib = fibo.fib
fib(500)

0 1 1 2 3 5 8 13 21 34 55 89 144 233 377 


模块可以包含可执行的语句及函数定义
- 语句用于初始化模块
    - 仅在模块 *第一次* 在 import 语句中被导入时执行
        - 实际上，函数定义也是“被执行”的“语句”
        - 模块级函数定义的执行在将函数名写入模块的全局符号表中
    - 当文件被当做脚本执行时，也会执行
    
每个模块都有自己的私有符号表
- 用作模块定义的所有函数的全局符号表
- 访问模块的全局变量 `modname.itemname`

模块可以导入其他模块
- 被导入的模块名存放在导入模块的全局符号表中

[import](https://docs.python.org/zh-cn/3/reference/simple_stmts.html#import) 语句变体
- 把名字从被调模块中导入到当前模块的符号表
    - 不会把被调模块名引入到局部变量表
- 导入模块内定义的所有名称
    - 导入所有非以下划线（`_`）开头的名称
    - 在解释器中引入了一组未知的名称，可能会覆盖已定义的函数或变量等

模块名之后带有 as
- 跟在 as 之后的名称将直接绑定到所导入的模块
- 使用 [from](https://docs.python.org/zh-cn/3/reference/simple_stmts.html#from) 时也可以使用
- 别名（alias）

每个模块在每个解释器会话中只被导入一次
- 更改模块必须重新启动解释器
- 如果要交互式地测试模块，使用 [importlib.reload()](https://docs.python.org/zh-cn/3/library/importlib.html#importlib.reload)
    - 例如 `import importlib; importlib.reload(modulename)`

In [5]:
# 导入函数名
from fibo import fib, fib2
fib(500)

0 1 1 2 3 5 8 13 21 34 55 89 144 233 377 


In [6]:
# 导入所有名称
from fibo import *
fib(500)

0 1 1 2 3 5 8 13 21 34 55 89 144 233 377 


In [7]:
# 绑定名称，别名
import fibo as fib
fib.fib(500)

0 1 1 2 3 5 8 13 21 34 55 89 144 233 377 


In [8]:
# 绑定名称，别名
from fibo import fib as fibonacci
fibonacci(500)

0 1 1 2 3 5 8 13 21 34 55 89 144 233 377 


### 以脚本方式执行模块

模块的代码会被执行
- `__name__` 被赋值为 `"__main__"`

```bash
python fibo.py <arguments>
```

在模块末尾添加以下代码
- 既可以当做脚本，又可以当做模块
- 只有在当模块以 main 文件的方式执行时才会运行
    - 导入模块不执行
- 为模块提供用户接口 / 用于测试
    - 以脚本的方式运行模块从而执行一些测试套件

```python
if __name__ == "__main__":
    import sys
    fib(int(sys.argv[1]))
```

### 模块搜索路径

1. 内置模块

2. [sys.path](https://docs.python.org/zh-cn/3/library/sys.html#sys.path) 变量给出的目录列表

初始包含以下目录地址：
- 包含输入脚本的目录（或未指定文件时的当前目录）
- [PYTHONPATH](https://docs.python.org/zh-cn/3/using/cmdline.html#envvar-PYTHONPATH) （一个包含目录名称的列表，和 shell 变量 PATH 有一样的语法）
- 取决于安装的默认设置

> 在支持符号链接的文件系统上，包含输入脚本的目录是在追加符号链接后才计算出来的。换句话说，包含符号链接的目录并没有被添加到模块的搜索路径上。

初始化后，Python 程序可以更改 sys.path
- 包含正在运行脚本的文件目录被放在搜索路径的开头处，在标准库路径之前
- 将加载该目录中的脚本，而不是标准库中的同名模块
- 除非有意更换，否则这是错误

### “编译过的” Python 文件

为了加速模块载入，Python 在 `__pycache__` 目录里缓存了每个模块的编译后版本
- 名称为 `module.version.pyc`
    - 其中名称中的版本字段对编译文件的格式进行编码
        - 一般使用 Python 版本号
    - 此命名约定允许来自不同发行版和不同版本的 Python 的已编译模块共存
    
例如，在 CPython 版本 3.3 中，spam.py 的编译版本将被缓存为  `__pycache__/spam.cpython-33.pyc`

Python 根据编译版本检查源的修改日期
- 查看是否已过期并需要重新编译
- 完全自动化
- 编译的模块与平台无关
    - 可以在具有不同体系结构的系统之间共享相同的库
    
Python 在两种情况下不会检查缓存
- 从命令行直接载入的模块
    - 重新编译并且不存储编译结果
- 没有源模块
    - 为了支持无源文件（仅编译）发行版本，编译模块必须是在源目录下，并且绝对不能有源模块
    
给专业人士的一些小建议:
- 可以在 Python 命令中使用 [-O](https://docs.python.org/zh-cn/3/using/cmdline.html#cmdoption-o) 或者 [-OO](https://docs.python.org/zh-cn/3/using/cmdline.html#cmdoption-oo) 开关，以减小编译后模块的大小
    - `-O` 开关去除断言语句
    - `-OO` 开关同时去除断言语句和 `__doc__` 字符串
    - 由于有些程序可能依赖于这些，应当只在清楚自己在做什么时才使用这个选项
    - “优化过的”模块有一个 `opt-` 标签并且通常小些
- 一个从 .pyc 文件读出的程序并不会比它从 .py 读出时运行的更快，.pyc 文件唯一快的地方在于载入速度
- [compileall](https://docs.python.org/zh-cn/3/library/compileall.html#module-compileall) 模块可以为一个目录下的所有模块创建 .pyc 文件
- 关于这个过程，[PEP 3147](https://www.python.org/dev/peps/pep-3147/) 中有更多细节，包括一个决策流程图

## 标准模块

标准模块库
- [Python 标准库](https://docs.python.org/zh-cn/3/library/index.html)

一些模块内置于解释器
- 提供对不属于语言核心但仍然内置的操作的访问
    - 提高效率
    - 提供对系统调用等操作系统原语的访问
- 这些模块的集合是配置选项
    - 取决于底层平台
    - [winreg](https://docs.python.org/zh-cn/3/library/winreg.html#module-winreg) 模块只在 Windows 操作系统上提供

[sys](https://docs.python.org/zh-cn/3/library/sys.html#module-sys) 模块
- 内嵌于所有 Python 解释器
- `sys.ps1`：首要提示符
- `sys.ps2`：次要提示符
- `sys.path`：解释器的模块搜索路径
    - 初始化为环境变量 [PYTHONPATH](https://docs.python.org/zh-cn/3/using/cmdline.html#envvar-PYTHONPATH) 获取的默认路径
    - PYTHONPATH 未设置，使用内置默认路径初始化

In [9]:
import sys
sys.ps1    # 解释器的首要提示符

'In : '

In [10]:
sys.ps2    # 解释器的次要提示符

'...: '

In [11]:
# 修改首要提示符
sys.ps1 = 'C> '
sys.ps1

'C> '

In [12]:
import sys
sys.path    # 解释器的模块搜索路径

['D:\\Study\\Github\\Study-Notes\\The Python Tutorial',
 'd:\\study\\github\\study-notes\\myenv\\scripts\\python38.zip',
 'c:\\python38\\DLLs',
 'c:\\python38\\lib',
 'c:\\python38',
 'd:\\study\\github\\study-notes\\myenv',
 '',
 'd:\\study\\github\\study-notes\\myenv\\lib\\site-packages',
 'd:\\study\\github\\study-notes\\myenv\\lib\\site-packages\\win32',
 'd:\\study\\github\\study-notes\\myenv\\lib\\site-packages\\win32\\lib',
 'd:\\study\\github\\study-notes\\myenv\\lib\\site-packages\\Pythonwin',
 'd:\\study\\github\\study-notes\\myenv\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\linki\\.ipython']

In [13]:
# 添加模块搜索路径
sys.path.append('/ufs/guido/lib/python')
sys.path

['D:\\Study\\Github\\Study-Notes\\The Python Tutorial',
 'd:\\study\\github\\study-notes\\myenv\\scripts\\python38.zip',
 'c:\\python38\\DLLs',
 'c:\\python38\\lib',
 'c:\\python38',
 'd:\\study\\github\\study-notes\\myenv',
 '',
 'd:\\study\\github\\study-notes\\myenv\\lib\\site-packages',
 'd:\\study\\github\\study-notes\\myenv\\lib\\site-packages\\win32',
 'd:\\study\\github\\study-notes\\myenv\\lib\\site-packages\\win32\\lib',
 'd:\\study\\github\\study-notes\\myenv\\lib\\site-packages\\Pythonwin',
 'd:\\study\\github\\study-notes\\myenv\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\linki\\.ipython',
 '/ufs/guido/lib/python']

## [dir()](https://docs.python.org/zh-cn/3/library/functions.html#dir) 函数

内置函数 `dir()`
- 查找模块定义的名称
- 返回排序过的字符串列表
- 列出所有类型的名称：变量，模块，函数，等等

内置函数和变量
- 定义在标准模块 [builtins](https://docs.python.org/zh-cn/3/library/builtins.html#module-builtins) 中

In [14]:
import fibo, sys
dir(fibo)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'fib',
 'fib2']

In [15]:
dir(sys)  

['__breakpointhook__',
 '__displayhook__',
 '__doc__',
 '__excepthook__',
 '__interactivehook__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '__stderr__',
 '__stdin__',
 '__stdout__',
 '__unraisablehook__',
 '_base_executable',
 '_clear_type_cache',
 '_current_frames',
 '_debugmallocstats',
 '_enablelegacywindowsfsencoding',
 '_framework',
 '_getframe',
 '_git',
 '_home',
 '_xoptions',
 'addaudithook',
 'api_version',
 'argv',
 'audit',
 'base_exec_prefix',
 'base_prefix',
 'breakpointhook',
 'builtin_module_names',
 'byteorder',
 'call_tracing',
 'callstats',
 'copyright',
 'displayhook',
 'dllhandle',
 'dont_write_bytecode',
 'exc_info',
 'excepthook',
 'exec_prefix',
 'executable',
 'exit',
 'flags',
 'float_info',
 'float_repr_style',
 'get_asyncgen_hooks',
 'get_coroutine_origin_tracking_depth',
 'getallocatedblocks',
 'getcheckinterval',
 'getdefaultencoding',
 'getfilesystemencodeerrors',
 'getfilesystemencoding',
 'getprofile',
 'getrecursionlimit',
 'getrefcount

In [16]:
# 不使用参数，列出当前定义的名称
a = [1, 2, 3, 4, 5]
import fibo
fib = fibo.fib
dir()

['In',
 'Out',
 '_',
 '_10',
 '_11',
 '_12',
 '_13',
 '_14',
 '_15',
 '_2',
 '_3',
 '_9',
 '__',
 '___',
 '__builtin__',
 '__builtins__',
 '__doc__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_dh',
 '_i',
 '_i1',
 '_i10',
 '_i11',
 '_i12',
 '_i13',
 '_i14',
 '_i15',
 '_i16',
 '_i2',
 '_i3',
 '_i4',
 '_i5',
 '_i6',
 '_i7',
 '_i8',
 '_i9',
 '_ih',
 '_ii',
 '_iii',
 '_oh',
 'a',
 'exit',
 'fib',
 'fib2',
 'fibo',
 'fibonacci',
 'get_ipython',
 'quit',
 'sys']

In [17]:
# 内置函数和变量的名称
import builtins
dir(builtins)

['ArithmeticError',
 'AssertionError',
 'AttributeError',
 'BaseException',
 'BlockingIOError',
 'BrokenPipeError',
 'BufferError',
 'BytesWarning',
 'ChildProcessError',
 'ConnectionAbortedError',
 'ConnectionError',
 'ConnectionRefusedError',
 'ConnectionResetError',
 'DeprecationWarning',
 'EOFError',
 'Ellipsis',
 'EnvironmentError',
 'Exception',
 'False',
 'FileExistsError',
 'FileNotFoundError',
 'FloatingPointError',
 'FutureWarning',
 'GeneratorExit',
 'IOError',
 'ImportError',
 'ImportWarning',
 'IndentationError',
 'IndexError',
 'InterruptedError',
 'IsADirectoryError',
 'KeyError',
 'KeyboardInterrupt',
 'LookupError',
 'MemoryError',
 'ModuleNotFoundError',
 'NameError',
 'None',
 'NotADirectoryError',
 'NotImplemented',
 'NotImplementedError',
 'OSError',
 'OverflowError',
 'PendingDeprecationWarning',
 'PermissionError',
 'ProcessLookupError',
 'RecursionError',
 'ReferenceError',
 'ResourceWarning',
 'RuntimeError',
 'RuntimeWarning',
 'StopAsyncIteration',
 'StopIter

## 包

通过“带点号的模块名”来构造 Python 模块命名空间的方法
- `A.B` 表示 `A` 包中名为 `B` 的子模块

可能的包结构

```
# 统一处理声音文件和声音数据

sound/                          Top-level package
      __init__.py               Initialize the sound package
      formats/                  Subpackage for file format conversions
              __init__.py
              wavread.py
              wavwrite.py
              aiffread.py
              aiffwrite.py
              auread.py
              auwrite.py
              ...
      effects/                  Subpackage for sound effects
              __init__.py
              echo.py
              surround.py
              reverse.py
              ...
      filters/                  Subpackage for filters
              __init__.py
              equalizer.py
              vocoder.py
              karaoke.py
              ...
```

导入包时，Python 搜索 `sys.path` 里的目录，查找包的子目录

必须有 `__init__.py` 文件才能让 Python 将包含该文件的目录当作包
- 防止具有通常名称（例如 `string`）的目录隐藏之后在模块搜索路径上出现的有效模块
- 在最简单的情况下，`__init__.py` 可以只是一个空文件
- 也可以执行包的初始化代码或设置 `__all__` 变量

```python
# 从包中导入单个模块
import sound.effects.echo    # 加载子模块
# 引用时必须使用全名
sound.effects.echo.echofilter(input, output, delay=0.7, atten=4)

# 导入子模块
from sound.effects import echo
# 无需包前缀
echo.echofilter(input, output, delay=0.7, atten=4)

# 直接导入所需的函数或变量
from sound.effects.echo import echofilter    # 加载子模块 echo
echofilter(input, output, delay=0.7, atten=4)
```

`from package import item`
- 报的子模块（子包）
- 包中定义的其他名称
    - 函数、类、变量
- `import` 语句首先测试是否在包中定义了 item
- 如果没有，则假定它是一个模块并尝试加载它
- 如果找不到它，则引发 [ImportError](https://docs.python.org/zh-cn/3/library/exceptions.html#ImportError) 异常

`import item.subitem.subsubitem`
- 除了最后一项之外的每一项都必须是一个包
- 最后一项可以是模块或包
    - 不能是前一项中定义的类、函数、变量

### 包的显式索引

`import` 语句规范
- 如果一个包的 `__init__.py` 代码定义了一个名为 `__all__` 的列表，它会被视为在遇到 `from package import *` 时应该导入的模块名列表
- 在发布该包的新版本时，包作者可以决定是否让此列表保持更新
- 包作者如果认为从他们的包中导入 * 的操作没有必要被使用，也可以决定不支持此列表

文件 `sound/effects/__init__.py` 可以包含以下代码
- `from sound.effects import *` 将导入 sound 包的三个命名子模块

```python
__all__ = ["echo", "surround", "reverse"]
```

如果没有定义 `__all__`
- `from sound.effects import *` 语句不会从包 sound.effects 中导入所有子模块到当前命名空间
- 只确保导入了包 sound.effects （可能运行任何在 `__init__.py` 中的初始化代码）
- 然后导入包中定义的任何名称
    - `__init__.py` 定义的任何名称（以及显式加载的子模块）
    - 由之前的 import 语句显式加载的包的任何子模块

```python
import sound.effects.echo
import sound.effects.surround
# 将 echo 和 surround 模块导入到当前命名空间
# 因为它们定义在 sound.effects 包中
from sound.effects import *
```

`from package import specific_submodule`
- 推荐的表示方式
- 除非导入的模块需要使用来自不同包的同名子模块

### 构造子包

使用绝对导入引用兄弟包的子模块
- Python 应用程序主模块的模块必须始终使用绝对导入
    - 主模块的名称总是 `"__main__"`

使用 import 语句的 `from module import name` 形式编写相对导入
- 使用前导点表示相对导入中涉及的当前包和父包
- 相对导入基于当前模块的名称进行导入

```python
from . import echo
from .. import formats
from ..filters import equalizer
```

### 多个目录中的包

包的特殊属性 [\_\_path\_\_](https://docs.python.org/zh-cn/3/reference/import.html#__path__)
- 初始化为一个列表
    - 包含在执行该文件中的代码之前保存包的文件 `__init__.py` 的目录的名称
- 可以修改
    - 影响将来对包中包含的模块和子包的搜索
- 可用于扩展程序包中的模块集

---

## 格式化输出

1. [格式化字符串字面值](https://docs.python.org/zh-cn/3/tutorial/inputoutput.html#tut-f-strings)
- 在字符串的开始引号或三引号前加上一个 `f` 或 `F`
- 可以在 `{` 和 `}` 字符之间写可引用的变量或字面值的 Python 表达式

In [18]:
year = 2016
event = 'Referendum'
f'Results of the {year} {event}'    # 引用变量

'Results of the 2016 Referendum'

2. 字符串的 [str.format()](https://docs.python.org/zh-cn/3/library/stdtypes.html#str.format) 方法
- 使用 `{` 和 `}` 标记变量将被替换的位置
- 可以提供详细的格式化指令
- 需要提供格式化的信息

In [19]:
yes_votes = 42_572_654
no_votes = 43_132_495
percentage = yes_votes / (yes_votes + no_votes)
'{:-9} YES votes  {:2.2%}'.format(yes_votes, percentage)

' 42572654 YES votes  49.67%'

3. 字符串切片和连接操作
- 字符串类型有一些方法可以执行将字符串填充到给定列宽的操作

In [20]:
s = "Hello, World"
s[:7] + "Python"

'Hello, Python'

4. 快速显示（将任何值转化为字符串）
- [repr()](https://docs.python.org/zh-cn/3/library/functions.html#repr)
    - 用于生成解释器可读的表示
    - 如果没有等效的语法，引发 [SynatxError](https://docs.python.org/zh-cn/3/library/exceptions.html#SyntaxError) 异常
- [str()](https://docs.python.org/zh-cn/3/library/stdtypes.html#str)
    - 返回人类可读的值的表示
    - 对于没有人类可读性的表示的对象，将返回和 `repr()` 一样的值
- 很多值使用任一函数都具有相同的表示
    - 比如数字或类似列表和字典的结构
- 特殊的是字符串有两个不同的表示

In [21]:
s = 'Hello, world.'
str(s)

'Hello, world.'

In [22]:
repr(s)

"'Hello, world.'"

In [23]:
str(1/7)

'0.14285714285714285'

In [24]:
# 数字
x = 10 * 3.25
y = 200 * 200
s = 'The value of x is ' + repr(x) + ', and y is ' + repr(y) + '...'
print(s)

The value of x is 32.5, and y is 40000...


In [25]:
# The repr() of a string adds string quotes and backslashes:
hello = 'hello, world\n'
hellos = repr(hello)    # 字符串引号和反斜杠
print(hellos)

'hello, world\n'


In [26]:
# The argument to repr() may be any Python object:
repr((x, y, ('spam', 'eggs')))    # 任意对象

"(32.5, 40000, ('spam', 'eggs'))"

5. [strings](https://docs.python.org/zh-cn/3/library/string.html#module-string) 模块包含一个 [Template](https://docs.python.org/zh-cn/3/library/string.html#string.Template) 类
- 使用类似 `$x` 的占位符，并用字典中的值替换
- 对格式的控制较少

In [27]:
from string import Template
s = Template('$who likes $what')
s.substitute(who='jck', what='cakes')    # 执行模板替换

'jck likes cakes'

### 格式化字符串文字

简称 f-字符串
- 在字符串前加上 `f` 或 `F` 并将表达式写成 `{expression}` 来在字符串中包含 Python 表达式的值
- 可选的格式说明符可以跟在表达式后面
    - 在 `:` 后传递一个整数可以指定最小字符宽度
- 其他的修饰符可用于在格式化之前转换值
    - `!a`：[ascii()](https://docs.python.org/zh-cn/3/library/functions.html#ascii)
    - `!s`：[str()](https://docs.python.org/zh-cn/3/library/stdtypes.html#str)
    - `!r`：[repr()](https://docs.python.org/zh-cn/3/library/functions.html#repr)
- 格式规范参考：[格式规格迷你语言](https://docs.python.org/zh-cn/3/library/string.html#formatspec)

In [28]:
# 保留小数点后三位
import math
print(f'The value of pi is approximately {math.pi:.3f}.')

The value of pi is approximately 3.142.


In [29]:
table = {'Sjoerd': 4127, 'Jack': 4098, 'Dcab': 7678}
for name, phone in table.items():
    print(f'{name:10} ==> {phone:10d}')    # 最小字符宽度

Sjoerd     ==>       4127
Jack       ==>       4098
Dcab       ==>       7678


In [30]:
animals = 'eels'
print(f'My hovercraft is full of {animals}.')
# 格式化之前转换值
print(f'My hovercraft is full of {animals!r}.')

My hovercraft is full of eels.
My hovercraft is full of 'eels'.


### 字符串的 [format()](https://docs.python.org/zh-cn/3/library/stdtypes.html#str.format) 方法

对象
- 花括号和其中的字符（称为格式字段）

位置
- 花括号中的数字

使用关键字参数
- 使用参数的名称引用它们的值

非常长的格式字符串
- 按名称引用变量
    - 通过传递字典并使用方括号访问键 `[]`
    - 通过使用 `**` 符号将字典作为关键字参数传递
    - 使用内置函数 [vars()](https://docs.python.org/zh-cn/3/library/functions.html#vars) ，返回包含所有局部变量的字典
    
[格式字符串语法](https://docs.python.org/zh-cn/3/library/string.html#formatstrings)

In [31]:
print('We are the {} who say "{}!"'.format('knights', 'Ni'))

We are the knights who say "Ni!"


In [32]:
print('{0} and {1}'.format('spam', 'eggs'))
print('{1} and {0}'.format('spam', 'eggs'))

spam and eggs
eggs and spam


In [33]:
# 关键字参数
print('This {food} is {adjective}.'.format(
      food='spam', adjective='absolutely horrible'))

This spam is absolutely horrible.


In [34]:
# 位置和关键字参数可以任意组合
print('The story of {0}, {1}, and {other}.'.format('Bill', 'Manfred',
                                                    other='Georg'))

The story of Bill, Manfred, and Georg.


In [35]:
# 方括号访问
table = {'Sjoerd': 4127, 'Jack': 4098, 'Dcab': 8637678}
print('Jack: {0[Jack]:d}; Sjoerd: {0[Sjoerd]:d}; '
      'Dcab: {0[Dcab]:d}'.format(table))

Jack: 4098; Sjoerd: 4127; Dcab: 8637678


In [36]:
# 解包访问
table = {'Sjoerd': 4127, 'Jack': 4098, 'Dcab': 8637678}
print('Jack: {Jack:d}; Sjoerd: {Sjoerd:d}; Dcab: {Dcab:d}'.format(**table))

Jack: 4098; Sjoerd: 4127; Dcab: 8637678


In [37]:
# 使用内置函数 vars()
for x in range(1, 11):
    print('{0:2d} {1:3d} {2:4d}'.format(x, x*x, x*x*x))

 1   1    1
 2   4    8
 3   9   27
 4  16   64
 5  25  125
 6  36  216
 7  49  343
 8  64  512
 9  81  729
10 100 1000


### 手动格式化字符串

[str.rjust()](https://docs.python.org/zh-cn/3/library/stdtypes.html#str.rjust)、[str,ljust()](https://docs.python.org/zh-cn/3/library/stdtypes.html#str.ljust)、[str.center()](https://docs.python.org/zh-cn/3/library/stdtypes.html#str.center)
- 通过填充空格来对给定宽度的字段中的字符串进行对齐
- 返回新的字符串

[str.zfill()](https://docs.python.org/zh-cn/3/library/stdtypes.html#str.zfill)
- 在数字字符串的左边填充零
- 能够识别正负号

字符串插值
- 给定 `'string' % values` ，则 `string` 中的 `%` 实例会以零个或多个 `values` 元素替换
- [printf 风格的字符串格式化](https://docs.python.org/zh-cn/3/library/stdtypes.html#old-string-formatting)

In [38]:
for x in range(1, 11):
    # 通过空格添加列之间的空格
    print(repr(x).rjust(2), repr(x*x).rjust(3), end=' ')
    # str.rjust() 在左侧填充空格来对给定宽度的字段中的字符串进行右对齐
    print(repr(x*x*x).rjust(4))

 1   1    1
 2   4    8
 3   9   27
 4  16   64
 5  25  125
 6  36  216
 7  49  343
 8  64  512
 9  81  729
10 100 1000


In [39]:
# 填充 0
'12'.zfill(5)

'00012'

In [40]:
# 识别负号
'-3.14'.zfill(7)

'-003.14'

In [41]:
# 宽度小于字符串长度，返回原字符串的副本
'3.14159265359'.zfill(5)

'3.14159265359'

In [42]:
# 字符串插值
import math
print('The value of pi is approximately %5.3f.' % math.pi)

The value of pi is approximately 3.142.


## 读写文件

[open()](https://docs.python.org/zh-cn/3/library/functions.html#open) 返回一个 [file object](https://docs.python.org/zh-cn/3/glossary.html#term-file-object)

常用参数：`open(filename, mode)`
- filename：包含文件名的字符串
- mode：包含一些描述文件使用方式的字符，默认为 `r`
    - `r`：只能读取
    - `w`：只能写入
    - `a`：追加内容
        - 任何写入的数据会自动添加到文件的末尾
    - `r+`：读写
    - `b`：二进制模式打开文件

文本模式（text mode）
- 从文件中读取或写入字符串时，都会以指定的编码方式进行编码
    - 如果未指定编码格式，默认值与平台相关
- 读取时默认把平台特定的行结束符转换为 `\n`
    - Unix 上的 `\n` ， Windows 上的 `\r\n`
- 写入时把 `\n` 转换回平台特定的结束符

二进制模式（binary mode）
- 数据是以字节对象的形式进行读写的
- 用于所有不包含文本的文件

[with](https://docs.python.org/zh-cn/3/reference/compound_stmts.html#with) 关键字
- 子句体结束后文件会正确关闭，即使在某个时刻引发了异常
- 相比等效的 [try](https://docs.python.org/zh-cn/3/reference/compound_stmts.html#tryffi)-[finally](https://docs.python.org/zh-cn/3/reference/compound_stmts.html#finally) 代码块更简洁

不使用 with 关键字
- 必须使用 `f.close()` 关闭文件并立即释放它所使用的所有系统资源
- 如果没有显式地关闭文件
    - Python 的垃圾回收器最终将销毁该对象并关闭打开的文件，但这个文件可能会保持打开状态一段时间
    - 不同的 Python 实现会在不同的时间进行清理

In [43]:
# 打开文件，子句体结束后自动关闭
with open('workfile') as f:
    read_data = f.read()

# We can check that the file has been automatically closed.
f.closed

True

In [44]:
# 通过 with 语句或调用 f.close() 关闭文件对象后，尝试使用该文件对象将自动失败
f.close()
f.read()

ValueError: I/O operation on closed file.

### 文件对象的方法

假定已创建名为 `f` 的文件对象

读取文件
- `f.read(size)`
    - 读取一些数据并将其作为字符串（文本模式）或字节串对象（二进制模式）返回
    - size 可选
        - 省略或为负数时返回整个文件的内容
        - 返回至多 size 个字符或字节
    - 到达文件末尾，返回空字符串 `''`
- `f,readline()`
    - 从文件中读取一行
    - 保留字符串末尾的 `\n`
        - 如果文件不以换行符结尾，则在文件的最后一行省略
    - 空行使用 `\n` 表示
    - 到达文件末尾，返回空字符串 `''`
- `f.readlines()`
    - 以列表的形式读取文件中的所有行
    - 等价于 `list(f)`

写入文件
- `f.write(string)`
    - 把 string 的内容写到文件中，返回写入的字符数
- 在写入其他类型的对象之前，需要先把它们转化为字符串（文本模式）或者字节对象（二进制模式）

In [45]:
# 读取整个文件的内容
f = open("workfile", "r")
f.read()

'This is the first line of the file.\nSecond line of the file.'

In [46]:
# 到达文件末尾
f.read()

''

In [47]:
# 按行读取
f.close()
f = open("workfile", "r")
f.readline()    # 第一行

'This is the first line of the file.\n'

In [48]:
f.readline()    # 第二行

'Second line of the file.'

In [49]:
f.readline()    # 到达文件末尾

''

In [50]:
# 按行循环遍历
f.close()
f = open("workfile", "r")
for line in f:
    print(line, end='')

This is the first line of the file.
Second line of the file.

In [51]:
# 写入文件
f.close()
f = open("workfile", "a")
f.write('This is the test line.\n')

23

In [52]:
# 写入元组
f.close()
f = open("workfile", "a")
value = ('the answer', 42)
s = str(value)  # convert the tuple to string
f.write(s)

18

`f.tell()` 返回文件对象在文件中的当前位置
- 二进制模式下，从文件开始的字节数
- 文本模式下的意义不明的数字

`f.seek(offset, whence)` 改变文件对象的位置
- 通过向一个参考点添加 offset 来计算位置
- 参考点由 whence 参数指定，默认值为 0
    - 0 表示从文件开头开始
    - 1 表示使用当前文件位置
    - 2 表示从文件末尾开始
- 文本模式只允许相对文件开头搜索
    - 唯一有效的 offset 值是那些能从 `f.tell()` 中返回的值或零
    - 其他 offset 值都会产生未定义的行为

In [53]:
f.close()
f = open('workfile2', 'rb+')    # 二进制模式打开文件
f.write(b'0123456789abcdef')   # 从头开始，覆盖写入

16

In [54]:
# 第 6 个字节
f.seek(5)      # Go to the 6th byte in the file

5

In [55]:
# 读取 1 字节
f.read(1)

b'5'

In [56]:
# 倒数第三个字节
f.seek(-3, 2)  # Go to the 3rd byte before the end

13

In [57]:
f.read(1)

b'd'

In [58]:
# 关闭文件
f.close()

### 使用 [json](https://docs.python.org/zh-cn/3/library/json.html#module-json) 保存结构化数据

[JSON (JavaScript Object Notation)](https://www.json.org/json-en.html)

json 标准模块
- 序列化（serializing）：采用 Python 数据层次结构，转化为字符串表示形式
- 反序列化（deserializing）：从字符串表示中重建数据

>  JSON 格式通常被现代应用程序用于数据交换。

[pickle](https://docs.python.org/zh-cn/3/library/pickle.html#module-pickle) 封存模块

> 与 JSON 不同，pickle 是一种允许对任意复杂 Python 对象进行序列化的协议。因此是 Python 特有的，不能用于与其他语言编写的应用程序通信。默认情况下它是不安全的：如果数据是由熟练的攻击者精心设计的，则反序列化来自不受信任来源的 pickle 数据可能可以执行任意代码。

In [59]:
# 查看 JSON 字符串表示
import json
json.dumps([1, 'simple', 'list'])

'[1, "simple", "list"]'

[json.dumps()](https://docs.python.org/zh-cn/3/library/json.html#json.dumps)：将对象序列化为 JSON 格式的字符串

[json.dump()](https://docs.python.org/zh-cn/3/library/json.html#json.dump)：将对象序列化为 [text file](https://docs.python.org/zh-cn/3/glossary.html#term-text-file)

可以处理列表和字典
- 在 JSON 中序列化任意类的实例需要格外的努力

```python
# 对象 x
# 文本文件 f

json.dump(x, f)

x = json.load(f)    # 解码
```